In [ ]:
pip install langchain langchain-google-genai langchain-experimental transformers torch sentence-transformers faiss-cpu neo4j openai langchain langchain-community transformers openllm accelerate langchain-openai UnstructuredURLLoader json-repair unstructured

<h1>Importing Dependencies, Loading .env variables

In [1]:
import os
from dotenv import load_dotenv

from neo4j import GraphDatabase

from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph


from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader, UnstructuredURLLoader

from langchain_openai import ChatOpenAI


In [3]:
load_dotenv(dotenv_path=".env")

neo4j_uri = os.getenv("NEO4J_URI")
neo4j_pass = os.getenv("NEO4J_PASSWORD")
google_api_key = os.getenv("GOOGLE_API_KEY")

In [5]:
# Instance Check

driver = GraphDatabase.driver(neo4j_uri,auth=("neo4j",neo4j_pass))

def run_q(q):
    with driver.session() as session:
        result = session.run(q)
        return [r.data() for r in result]

<h2> Generate Graph

In [136]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=google_api_key,
    temperature=0.7,
)

In [155]:
endpoints = [
    "https://www.mosdac.gov.in/insat-3dr",
    "https://www.mosdac.gov.in/insat-3d",
    "https://www.mosdac.gov.in/kalpana-1",
    "https://www.mosdac.gov.in/insat-3a",
    "https://www.mosdac.gov.in/megha-tropiques",
    "https://www.mosdac.gov.in/saral-altika",
    "https://www.mosdac.gov.in/oceansat-2",
    "https://www.mosdac.gov.in/oceansat-3",
    "https://www.mosdac.gov.in/insat-3ds",
    "https://www.mosdac.gov.in/scatsat-1"
]

transformer = LLMGraphTransformer(llm=llm)

for end in endpoints:
    links = [end]
    for s in ["Introduction","Objectives","SpaceCraft","Payloads"]:
        links.append(end + f"-{s}")
    
    docs = UnstructuredURLLoader(urls=links).load()
    kg_docs = transformer.convert_to_graph_documents(docs)
    graph.add_graph_documents(kg_docs, baseEntityLabel=True, include_source=True)
            
        

<h2> Inference</h2>

In [128]:
graph = Neo4jGraph(
    url=neo4j_uri,
    username="neo4j",
    password=neo4j_pass
)

graph.refresh_schema()

chain = GraphCypherQAChain.from_llm(llm, graph=graph, verbose=True, allow_dangerous_requests=True)


In [163]:
response = chain.invoke("what are the objectives of megha tropiques")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s)-[:HAS_OBJECTIVE]->(o:Objective) WHERE s.id = 'Megha-Tropiques' RETURN o.id
Full Context:
[{'o.id': 'understand the life cycle of convective systems that influence the tropical weather and climate'}, {'o.id': 'study the role of convective systems in energy and moisture budget of the atmosphere in tropical regions'}]

> Finished chain.
{'query': 'what are the objectives of megha tropiques', 'result': 'The objectives of Megha-Tropiques are to understand the life cycle of convective systems that influence the tropical weather and climate, and to study the role of convective systems in energy and moisture budget of the atmosphere in tropical regions.'}


In [173]:
print(chain.invoke("tell me about the design of oceansat-3"))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Satellite {id: 'Oceansat-3'})
OPTIONAL MATCH (s)-[r]->(design_element)
WHERE type(r) IN [
    'HAS_INSTRUMENT', 'HAS_PAYLOAD', 'HAS_ORBIT_TYPE', 'USES_TECHNOLOGY',
    'HAS_CHARACTERISTIC', 'CONFIGURED_ON', 'HAS_SUBSYSTEM', 'USES_SPACECRAFT_BUS',
    'DEVELOPED_FROM', 'HAS_COMPONENT', 'HAS_PROPULSION_SYSTEM', 'HAS_DESIGN_LIFE',
    'HAS_OPERATIONAL_GOAL', 'USES_ATTITUDE_CONTROL_SYSTEM', 'USES_PROPULSION_SYSTEM',
    'USES_POWER_BUS', 'USES_SOLAR_PANEL'
]
RETURN s.id AS Satellite,
       COLLECT(DISTINCT {
           relationship: type(r),
           target_id: design_element.id,
           target_labels: labels(design_element)
       }) AS DesignDetails
Full Context:
[{'Satellite': 'Oceansat-3', 'DesignDetails': [{'target_labels': ['__Entity__', 'Instrument'], 'target_id': 'Sea Surface Temperature Monitor', 'relationship': 'HAS_INSTRUMENT'}, {'target_labels': ['__Entity__', 'Instrument', 'Payload'], 'target_id': 'A

In [74]:
print(chain.invoke("tell me about the product E06OCM_L2C_LAC_AD"))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product {id: 'E06OCM_L2C_LAC_AD'}) OPTIONAL MATCH (p)-[r]-(n) RETURN p, r, n
Full Context:
[{'p': {'id': 'E06OCM_L2C_LAC_AD', 'embedding': [-0.036945078521966934, -0.0883641168475151, -0.06253153830766678, -0.01790975034236908, 0.013010053895413876, 0.0021708293352276087, 0.02121014893054962, 0.11853482574224472, -0.017019039019942284, -0.012731427326798439, 0.13536342978477478, -0.1083293929696083, -0.09154032170772552, -0.009308191016316414, -0.04859107360243797, 0.01985546573996544, -0.10205056518316269, 0.02060863748192787, -0.02086961828172207, 0.10401985049247742, 0.037823498249053955, 0.024431049823760986, -0.03692740947008133, 0.025170497596263885, 0.01934307999908924, 0.00694165239110589, 0.0394647978246212, 0.06669629365205765, -0.010527470149099827, -0.073744036257267, 0.037595316767692566, 0.0008376260520890355, 0.06081332266330719, -0.006643485277891159, 0.062084097415208817, -0.010661475360393524, -0.